### Problem formulation

some text here
<br/><br/><br/>

In [47]:
import findspark
findspark.init()

In [48]:
import pyspark
from pyspark.sql import SparkSession

In [49]:
# imports here

import re
from datetime import datetime

In [50]:
import nltk
from nltk.corpus import stopwords

In [66]:
# global variables definition

#usa_training_tweets_collection_23_06
tweets_table = "usa_training_tweets_30_06.training_tweets_collection"

number_of_hours_to_get_topics = 2
frame_start_datetime = "Sun Jun 30 00:00:00 +0000 2019"
frame_finish_datetime = "Sun Jun 30 23:00:00 +0000 2019"

In [67]:
### we should put this block in the util functions file

# def str_to_tweeter_datetime():
#     ts2 = datetime.strptime(frame_start_datetime,'%a %b %d %H:%M:%S %z %Y')
#     ts = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(frame_start_datetime,'%a %b %d %H:%M:%S +0000 %Y'))

### Connecting to MongoDB

**Creating Spark connector**

In [68]:
spark = SparkSession.builder.appName("pipeline") \
    .config('spark.mongodb.input.uri', 'mongodb://localhost:27017/'+tweets_table) \
    .config('spark.mongodb.output.uri', 'mongodb://localhost:27017/'+tweets_table) \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.1') \
    .config('spark.mongodb.input.partitioner', 'MongoPaginateBySizePartitioner') \
    .getOrCreate()

## Huge bottleneck, isn't it?

**Creating Spark dataframe on top of DB**

In [69]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [70]:
# df.createOrReplaceTempView("temp")

In [71]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- contributors: null (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |-- dis

In [57]:
# Sample query
# sample = spark.sql('SELECT * FROM temp LIMIT 10')
# sample.collect()

# Data description (Женя)
how e collected it?

for what timeframe we have a date and for what region (NY/USA)<br/>
size of tweets that we have (for NY/USA)

### Data preprocessing

#### Selecting data from {{ frame_start_datetime }} to {{frame_finish_datetime}}

In [73]:
sample = spark.sql('SELECT * FROM root LIMIT 10')

AnalysisException: 'Table or view not found: root; line 1 pos 14'

In [59]:
StopWords = stopwords.words("english")

tokens = df.select('text').rdd \
    .map(lambda x: x[0]) \
    .map( lambda document: re.split(" ", document))          \
    .map( lambda word: [x for x in word if x.isalpha()])           \
    .map( lambda word: [x for x in word if len(x) > 3] )           \
    .map( lambda word: [x for x in word if x not in StopWords])    \


AnalysisException: "cannot resolve '`text`' given input columns: [];;\n'Project ['text]\n+- Relation[] MongoRelation(MongoRDD[14] at RDD at MongoRDD.scala:51,Some(StructType()))\n"

In [60]:
tokens.collect()

NameError: name 'tokens' is not defined

### Topic modeling (LDA)

### Hot topics in the USA from [Google trends](https://trends.google.com/trends/explore?geo=US)

### Conclusion